In [ ]:
import os
from datetime import datetime
import pandas as pd
from sqlalchemy import create_engine, text
import time
import numpy as np
import dotenv

dotenv.load_dotenv(".env.local")

from test_unit.tasks_unit1 import UnitTask1
from mrp_functions.base_tasks import TaskBase
from mrp_functions import logging
import re

np.__version__

part_group=os.environ['PART_GROUP']
contract_code=os.environ['CONTRACT_CODE']
mrp_run_date=os.environ['MRP_RUN_DATE']
msc_code=os.environ['MSC_CODE']
plant_code=os.environ['PLANT_CODE']
production_plan_id=int(os.environ['PRODUCTION_PLAN_ID'])
user_code=int(os.environ['USER_CODE'])
simulation=bool(int(os.environ['SIMULATION']))

tb = TaskBase()
tb.db_connection_health_check()

In [ ]:
# Get end date of production plan
end_date_of_prod_plan = tb.query_end_date_from_production_plan(production_plan_id)
if end_date_of_prod_plan is None:
    raise ValueError("Production Plan End Date is None!")
# Get list of all date from N to N+6 to end of production Plan
list_mrp_date = tb.query_list_date_from_Nplus1_to_Nplus6_definition(mrp_run_date, end_date_of_prod_plan)
print(f"Inventory Forecast From: {list_mrp_date[0]} To: {list_mrp_date[-1]}")

In [ ]:
# To query current Logical Inventory Quantity
log_inventory_frame = tb.query_inventory_log(mrp_run_date, plant_code, simulation)
logging.info(f"[{production_plan_id}_{mrp_run_date}_{plant_code}] "
             f"Current Inventory Size: {len(log_inventory_frame)}")
log_inventory_frame = log_inventory_frame.pivot_table(
            index="production_date",
            columns=["part_code", "part_color_code"],
            values="quantity",
            aggfunc='sum')
log_inventory_frame

In [ ]:
mrp_results_frame = tb.query_mrp_results(mrp_run_date, plant_code, production_plan_id, simulation)
logging.info(f"[{production_plan_id}_{mrp_run_date}_{plant_code}] "
             f"MRP Results Size: {len(mrp_results_frame)}")
mrp_results_frame = mrp_results_frame.pivot_table(
            index="production_date",
            columns=["part_code", "part_color_code"],
            values="quantity",
            aggfunc='sum')
mrp_results_frame

In [ ]:
merge_logical_parts_columns = mrp_results_frame.columns.tolist()
for _col in log_inventory_frame.columns:
    if _col not in merge_logical_parts_columns:
        merge_logical_parts_columns.append(_col)

merge_logical_parts_inv = pd.DataFrame(np.zeros((len(list_mrp_date), len(merge_logical_parts_columns))), index=list_mrp_date)
merge_logical_parts_inv.index.names = ['production_date']
merge_logical_parts_inv.columns = pd.MultiIndex.from_tuples(merge_logical_parts_columns, names=['part_code', 'part_color_code'])

merge_mrp_result_parts = (merge_logical_parts_inv + mrp_results_frame.fillna(0.0)).fillna(0.0)

if len(log_inventory_frame) == 0:
    merge_logical_parts_inv = merge_logical_parts_inv.fillna(0.0)
else:
    merge_logical_parts_inv = (merge_logical_parts_inv + log_inventory_frame).fillna(0.0)

merge_logical_parts_inv


In [ ]:
merge_mrp_result_parts

In [ ]:
final_logical_inventory = merge_logical_parts_inv - merge_mrp_result_parts.cumsum()
final_logical_inventory


In [ ]:
final_shortage_parts = final_logical_inventory.copy(deep=True)
final_shortage_parts = final_shortage_parts.applymap(lambda x: np.minimum(0.0, x))
final_shortage_parts = final_shortage_parts - final_shortage_parts.shift(1).copy(deep=True).fillna(0.0)
final_shortage_parts


In [ ]:
parent_start_timestamp = time.time()
# to save Logical Inventory to database
log_inventory = final_logical_inventory.loc[
    final_logical_inventory.index > datetime.date(datetime.strptime(mrp_run_date, '%Y-%m-%d'))].copy(deep=True)
if len(log_inventory) > 0:
    # Drop first row by selecting all rows from first row onwards
    log_inventory = log_inventory.loc[
        log_inventory.index > datetime.date(datetime.strptime(mrp_run_date, "%Y-%m-%d"))].copy(deep=True)
    log_inventory = log_inventory.stack().reset_index()
    log_inventory = log_inventory.set_index(['production_date', 'part_color_code']).stack().reset_index()
    log_inventory.rename(columns={log_inventory.columns[-1]: 'quantity'}, inplace=True)

    log_inventory[['plant_code']] = plant_code
    log_inventory[['created_by']] = user_code
    log_inventory[['updated_by']] = user_code

    logging.info(f"[{production_plan_id}_{mrp_run_date}_{plant_code}] "
                 f"Logical Inventory Size: {len(log_inventory)}")
    logging.debug(f"[{plant_code}] Logical Inventory:\n{log_inventory.to_dict(orient='records')}")
    tb.store_logical_inventory_results(log_inventory.to_dict(orient="records"),
                                       upsert=False, is_simulation=simulation)
    logging.info(f"[{production_plan_id}_{mrp_run_date}_{plant_code}] "
                 f"Calculate Logical Inventory finished after: {time.time() - parent_start_timestamp:.2f} sec")


In [ ]:
log_parts_shortage = final_shortage_parts.copy(deep=True)
if simulation:
    # to save Shortage Parts to database
    if log_parts_shortage is not None and len(log_parts_shortage) > 0:
        log_parts_shortage = log_parts_shortage.loc[
            log_parts_shortage.index > datetime.date(datetime.strptime(mrp_run_date, "%Y-%m-%d"))]
        log_parts_shortage = log_parts_shortage.stack().reset_index()
        log_parts_shortage = log_parts_shortage.set_index(
            ['production_date', 'part_color_code']).stack().reset_index()

        log_parts_shortage.rename(columns={log_parts_shortage.columns[-1]: 'quantity'}, inplace=True)
        # To ignore all positive or zero quantity
        log_parts_shortage = log_parts_shortage.loc[log_parts_shortage.quantity < 0].copy(deep=True)

        log_parts_shortage[['import_id']] = production_plan_id
        log_parts_shortage[['plant_code']] = plant_code
        log_parts_shortage[['created_by']] = user_code
        log_parts_shortage[['updated_by']] = user_code

        logging.info(f"[{production_plan_id}_{mrp_run_date}_{plant_code}] "
                     f"Shortage Parts Size: {len(log_parts_shortage)}")
        logging.debug(f"[{plant_code}] Shortage Parts:\n{log_parts_shortage.to_dict(orient='records')}")
        # to save Shortage Parts
        tb.store_shortage_parts_results(log_parts_shortage.to_dict(orient="records"))
        logging.info(f"[{production_plan_id}_{mrp_run_date}_{plant_code}] "
                     f"Store Shortage Parts finished after: {time.time() - parent_start_timestamp:.2f} sec")
